In [ ]:
from __future__ import division
%matplotlib inline
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
import h5py
import os
import six
from six.moves import range
import itertools
import matplotlib.pyplot as plt
import PIL
from tqdm import tqdm
from PIL import ImageOps
from PIL import ImageEnhance
import matplotlib.font_manager
from PIL import ImageDraw, ImageFont, ImageFilter
import tempfile
from datetime import datetime
import shutil

from deep_car.data import data_generator, discretize, continuous, augment_img, \
    augment_batch, batch_to_numpy, crop_batch, \
    get_steering_delta, get_steering_hist
from deep_car.model import Model

In [ ]:
h5py.version.version

In [ ]:
data_dir = '../data'
model_dir = '../data/model'
model_name = 'steering_mixture_prob_exp'
tmp_dir = '../tmp'

os.makedirs(tmp_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)
crop_size = (64, 48)

h5_train = h5py.File(os.path.join(data_dir, 'train.hdf5'))
h5_test = h5py.File(os.path.join(data_dir, 'test.hdf5'))


delta_discretize_min = -36 - 4.5
delta_discretize_max = 36 + 4.5
delta_discretize_buckets = 9

In [ ]:
len(h5_train.keys())

In [ ]:
print("{:25}| {:10}| {:30}".format("name", "dtype", "shape"))
print("-" * 40)
for name, dset in h5_train.items():
    print("{:25}| {:10}| {:30}".format(name, str(dset.dtype), str(dset.shape)))

In [ ]:
(delta_discretize_max - delta_discretize_min) / 9

In [ ]:
def data_generator(h5, batch_size=128, n_epoch=-1, shuffle=True, steering_distance_max=500):
    def get_steering(idx):
        idx = clip(idx)
        return np.array(steering[idx])

    def clip(x):
        return np.clip(x, 0, n-1)

    steering = np.array(h5['steering'])

    n = len(h5['image'])
    idx = np.arange(n)
    if n_epoch == -1:
        n_epoch = 1000000000

    for epoch in range(n_epoch):
        if shuffle:
            np.random.shuffle(idx)
        for b in range(0, n, batch_size):
            batch_idx = np.sort(idx[b:b+batch_size])            
            batch = {
                'image': h5['image'][batch_idx, :, :],
                'steering_abs': h5['steering'][batch_idx, :],
            }
            yield batch

In [ ]:
batch = next(data_generator(h5_train))
for name, arr in sorted(batch.items()):
    print("{:<17} | {:} ".format(name, arr.shape))

In [ ]:
batch['image'].min(), batch['image'].max(), 

#### number of seconds per epoch

In [ ]:
#%%timeit -n 3

#for batch in data_generator(n_epoch=1):
#    pass

In [ ]:
crop_size

In [ ]:
batch = next(data_generator(h5_train))

In [ ]:
images = [PIL.Image.fromarray(x) for x in batch["image"]]
img = images[0]
fig, axes = plt.subplots(4, 12, figsize=(20, 5))

for ax in axes[:1].flat:
    ax.imshow(np.array(img), cmap='gray', vmin=0, vmax=255)
    ax.set_xticks([])
    ax.set_yticks([])
for ax in axes[1:].flat:
    ax.imshow(np.array(augment_img(img)), cmap='gray', vmin=0, vmax=255)
    ax.set_xticks([])
    ax.set_yticks([])

In [ ]:
def batch_to_numpy(batch):                                                                                                                                                
    x = 2. * batch['image'] / 255. - 1                                          
    steering_abs = batch['steering_abs']                                                                              
    return x[:, :, :, np.newaxis], steering_abs

In [ ]:
batch_aug = augment_batch(batch)
x_image, x_steering_abs = batch_to_numpy(batch_aug)

for name, arr in [
    ('data', x_image),
    ('steering_abs', x_steering_abs),
]:
    print("{:30} | {:20} | {:10} | {:10}".format(name, str(arr.shape), float(arr.min()), float(arr.max())))


In [ ]:
y_delta_buckets = 2
gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

input_shape = [None, crop_size[1], crop_size[0], 1]
m = Model(input_shape)

init_op = tf.global_variables_initializer()
sess.run(init_op)

In [ ]:
history = {}
for name in ['steering_abs']:
    history[name] = []
    history["val_" + name] = []

In [ ]:
batch_size = 100
n_batches_per_epoch = len(h5_train['image']) // batch_size

tqdm_gen = tqdm(data_generator(h5_train, batch_size=batch_size, n_epoch=20))
running_loss = 'init'

saver = tf.train.Saver()
now = datetime.now()
save_dir = os.path.join(model_dir, model_name + "_" + now.isoformat())
os.makedirs(save_dir)

for i, batch in enumerate(tqdm_gen):
    x_image, x_steering_abs = batch_to_numpy(augment_batch(batch))
    steering_loss, _ = sess.run(
        [m.steering_abs_loss, m.opt_op], 
        feed_dict={
            m.image: x_image, 
            m.steering_abs_true: x_steering_abs,
            m.training: True,
    })
    
    history['steering_abs'].append(np.mean(steering_loss))
    batch_loss = np.mean(steering_loss)
    if running_loss == 'init':
        running_loss = batch_loss
    else:
        running_loss = 0.9*running_loss + 0.1*batch_loss
        
    if i % n_batches_per_epoch == 0:
        val_steering_abs = []
        for test_batch in data_generator(h5_test, batch_size=batch_size, n_epoch=1):
            x_image, x_steering_abs = batch_to_numpy(crop_batch(test_batch))
            steering_abs_loss = sess.run(
                [m.steering_abs_loss],
                feed_dict={
                    m.image: x_image, 
                    m.steering_abs_true: x_steering_abs,
                }
            )
            val_steering_abs.append(np.mean(steering_abs_loss))
            
        history['val_steering_abs'].append(np.mean(val_steering_abs))
            
    tqdm_gen.set_description('loss: {:.02f} - val_loss: {:.02f}'.format(running_loss, history['val_steering_abs'][-1]))

save_path = saver.save(sess, os.path.join(save_dir, model_name + ".ckpt"))
print("Saved model in: " + os.path.abspath(save_dir))

In [ ]:
save_path = saver.save(sess, os.path.join(save_dir, model_name + ".ckpt"))
print("Saved model in: " + os.path.abspath(save_dir))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

for name, hist in sorted(history.items()):
    if name.startswith("val"):
        ax.plot(np.arange(0, len(hist)*n_batches_per_epoch, n_batches_per_epoch), hist, label=name)
    else:
        ax.plot(history[name], label=name)
ax.legend()

In [ ]:
x_image.shape

In [ ]:
for test_batch in data_generator(h5_test, batch_size=batch_size, n_epoch=1):
    x_image, x_steering_abs, y_delta_disc, y_distance_to_next_steering = batch_to_numpy(crop_batch(test_batch))
    break
            
y_distance_prob, y_steering_prob = sess.run(
    [m.y_distance_prob, m.y_steering_prob], 
    feed_dict={m.image: x_image, m.steering_abs: x_steering_abs, m.y_distance_true: y_distance_to_next_steering})
nrows = 16
fig, axes = plt.subplots(ncols=2, nrows=nrows, figsize=(12, 2*nrows))
for i, ax in enumerate(axes[:, 1]):
    ax.bar(np.linspace(0, 1, len(y_distance_prob[i])), np.exp(y_distance_prob[i]), width=1/len(probs[i]))
    ax.vlines(y_distance_to_next_steering[i], 0, ax.get_yticks()[-1], color='r')
    axes[i, 0].imshow(x_image[i, :, :, 0], cmap='gray')
    
    axes[i, 0].set_title("{:0.3f}".format(float(y_steering_prob[i, 0])))
    axes[i, 0].set_xticks([])
    axes[i, 0].set_yticks([])

In [ ]:
rows = 10
cols = y_delta_prob_.shape[1]
bins = y_delta_prob_.shape[2]
fig, axes = plt.subplots(rows, cols+2, figsize=(2*cols, rows))

for i in range(rows):
    for j in range(cols+2):
        ax = axes[i, j]
        if j == 0:
            ax.imshow(X[i, :, :, 0], cmap='gray')
            continue
        if j == 1:
            ax.bar(np.arange(180) - 90, np.exp(probs[i]))
            continue
            
        j = j - 2
        ax.bar(np.arange(bins), y_delta_prob_[i, j])
        ax.set_ylim(0, 1)

for ax in axes.flat:
    ax.set_yticks([])
    ax.set_xticks([])

In [ ]:
def image_draw_info(img, steering_true, steering_pred, font=None):
    scale = 30
    pred_max = max(steering_pred)
    steering_pred = np.array([xi/pred_max * scale for xi in steering_pred])
    if font is None:
        fonts = matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')
        mono_fonts = [f for f in fonts if "mono" in f.lower() and 'bold' in f.lower()]
        mono_font = mono_fonts[0]
        font = ImageFont.truetype(mono_font, 18)
    
    
    img = img.resize((640,480))
    img= img.convert("RGBA")
    draw = ImageDraw.Draw(img)

    
    angle = steering_true + np.pi/2
    x = 1/2 * img.size[0]
    y = 90
    x_start = x - 200
    
    legend_color = "#ff0000"
    pred_color = "#0099ffaa"
    draw.text((10,10), "real:{}".format(steering_true), fill="#00ff00ff", font=font)
    draw.text(
        (img.size[0] - 300,10), 
        "best predict:{}-> {:.2f}".format(np.argmax(steering_pred) - 90,max(steering_pred)), 
        fill=pred_color,
        font=font
    )
    
    draw.text((x_start-5, y+10), "-90", fill=legend_color, font=font)
    draw.text((x-5, y+10), "0", fill=legend_color, font=font)
    draw.text((x-5+ 180, y+10), "90", fill=legend_color, font=font)
    for i,prob in enumerate(steering_pred):
        draw.line([x+2*(i-90), y, x+2*(i-90), y-prob], fill=pred_color, width=2)
    
    draw.line([x+2*steering_true, y, x+2*steering_true, y - scale], fill="#00ff00ff", width=2)                       
    del draw
    
    return img




In [ ]:
data_stream = data_generator(h5_train, batch_size=100, n_epoch=1, shuffle=False)
_ = next(data_stream)

In [ ]:
def extract_normalized_images_from(h5_data):
    x = 2.* h5_data['image'][:] / 255. - 1 
    y = h5_data['steering'][:]
    return x[:,6:-6,8:-8,np.newaxis],y

def get_chunks_from(h5_data, chunk_size):
    X,y = extract_normalized_images_from(h5_data)
    for i in range(0, len(X), chunk_size):
        yield X[i: i+chunk_size],y[i: i+chunk_size]
    
tmp = os.path.abspath('../tmp/')
h5_file = h5py.File(os.path.join(data_dir, 'test.hdf5'))
tmp_dir = tempfile.mkdtemp(dir=tmp)
predictions = []
for X,y in get_chunks_from(h5_file, 100):
    prediction = sess.run([y_pred], feed_dict={x:X, y_true:y})
    predictions.append(prediction[0])
predictions = np.concatenate(predictions)    


In [ ]:
img_fnames = []
video_dir = tempfile.mkdtemp(dir=tmp_dir)
print(video_dir)
idx = 0
for batch in tqdm(data_stream):
    X, s_hist, y_delta_, y_abs_ = batch_to_numpy(batch)
    X = X[:, 6:-6, 8:-8, :]
    probs, y_delta_prob_ = sess.run([y_abs_discr_prob, y_delta_prob], 
                                    feed_dict={x: X, steering_hist: s_hist})
    for i in range(len(X)):
        img = PIL.Image.fromarray((255*(X[i, :, :, 0]/2 + 0.5)).astype(np.uint8), 'L')
        img = image_draw_info(img, y_abs_[i, 0]/np.pi * 180, np.exp(probs[i]))
        img_fname = os.path.join(video_dir, "{:06d}.png".format(idx))
        img.save(img_fname)
        idx += 1
    
    if idx > 500:
        break

In [ ]:
def save_image(img, i):
    img_fname = os.path.join(tmp_dir, "{:06d}.png".format(i))
    img.save(img_fname)

try:
    for i,test_pred,test_real, img in zip(range(len(predictions)), predictions, h5_file['steering'][:], h5_file['image'][:]):
        img = PIL.Image.fromarray(img)
        image_draw_info(img_scale, test_real, test_pred)
        save_image(img_scale,i)
        
    video = ImageSequenceClip(tmp_dir, fps=24, with_mask=False)
    video.write_videofile("test_set_evaluation.webm", ffmpeg_params=['-b:v', '0', '-crf', '20'])
finally:        
    shutil.rmtree(tmp_dir)

In [ ]:
1 - 1